### **TLDR** 

 - **Dashboard**: 
  - Google Data Studio ([link](https://datastudio.google.com/reporting/2db6170e-9309-41c1-b9b9-04f5ed5173a8)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/carloseschholz/covid-da)). 
 - **Fontes**: 
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

# **COVID Dashboard**

## 1\. Contexto

   A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).
   A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil.

## 2\. Pacotes e bibliotecas

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

## 3\. Extração

In [2]:
# Vamos processar os dados de casos da universidade John Hopkins.

# O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.

cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv'
                    , sep=',')

cases.head()


In [7]:
# Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo.

def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)
    
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

In [8]:
# De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'],
                   axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = cases.append(case, ignore_index=True)
    
cases.query('Province_State == "Sao Paulo"').head()

In [10]:
# Agora vamos processar os dados de vacinação da universidade de Oxford.

vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',',
                       parse_dates=[3], infer_datetime_format=True)

vaccines.head()

In [11]:
# Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated',
                     'people_fully_vaccinated', 'total_boosters', 'date']]

vaccines.tail()

## 4\. Transformação

In [12]:
# Vamos manipular os dados da universidade John Hopkins para o dashboard. 
#O foco é em garantir uma boa granularidade e qualidade da base de dados.

cases.head()

In [ ]:
cases.shape

In [ ]:
cases.info()

In [ ]:
# Começamos com o nome das colunas.

cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

In [ ]:
# Ajustamos o nome dos estados.

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

In [ ]:
# Vamos então computar novas colunas para enriquecer a base de dados.

#Chaves temporais:

cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

# População estimada do estado:

cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

# Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.75:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status
  

for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = cases_.append(cases_per_state, ignore_index=True)

cases = cases_
cases_ = None

In [ ]:
#Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o type casting das colunas.

cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

In [ ]:
# Por fim, vamos reorganizar as colunas e conferir o resultado final.

cases = cases[['date', 'country', 'state', 'population', 'confirmed',
               'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend',
               'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 
               'month', 'year']]

cases.head(n=25)

In [ ]:
# Vamos manipular os dados  da  da universidade de Oxford para o dashboard.
# O foco é em garantir uma boa granularidade e qualidade da base de dados.

vaccines.head()

In [ ]:
vaccines.shape

In [ ]:
vaccines.info()

In [ ]:
# Vamos começar tratando os dados faltantes, a estratégia será a de preencher os buracos com o valor anterior válido mais próximo.

vaccines = vaccines.fillna(method='ffill')

In [ ]:
# Vamos também filtrar a base de dados de acordo com a coluna date para garantir que ambas as bases de dados tratam do mesmo período de tempo.

vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

In [ ]:
# Agora, vamos alterar o nome das colunas.

vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

In [ ]:
# Vamos então computar novas colunas para enriquecer a base de dados.

# Chaves temporais:

vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

# Dados relativos:

vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

In [ ]:
# Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o type casting das colunas.

vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

In [ ]:
# Por fim, vamos reorganizar as colunas e conferir o resultado final.

vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc',
                     'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

vaccines.tail()

## 5\. Carregamento

In [ ]:
cases.to_csv('./covid-cases.csv', sep=',', index=False)

vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)